In [2]:
import gzip
import pandas as pd
import numpy as np
from mnist import MNIST
import torch
from torch import nn
import copy
import torch.optim as optim

In [3]:
mndata = MNIST('Data')
#images, labels = mndata.load_training()
images, labels = mndata.load_testing()

#print(images)

In [ ]:
class GAN_Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 512)
        self.fc3 = nn.Linear(512, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
class Discriminator_Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [21]:
gunzip("Data/t*-ubyte.gz")

NameError: name 'gunzip' is not defined